In [1]:
%pip install --no-cache-dir git+https://github.com/konveyor/kai.git@main
%pip install python-dotenv

  Cloning https://github.com/konveyor/kai.git (to revision main) to /tmp/pip-req-build-ugco8xe2
  Running command git clone --filter=blob:none --quiet https://github.com/konveyor/kai.git /tmp/pip-req-build-ugco8xe2
  Resolved https://github.com/konveyor/kai.git to commit 7031fed36a1521a01a249351afa6ed82db92f32e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from kai.llm_interfacing.model_provider import ModelProvider
from kai.kai_config import KaiConfigModels, SupportedModelProviders
from dotenv import load_dotenv
load_dotenv()

model_provider = ModelProvider.from_config(KaiConfigModels(
  provider=SupportedModelProviders.CHAT_OPENAI,
  args={"model": "gpt-4o"}
))

/home/jonah/projects/kai-mcp-experiments/.venv/lib64/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
before_code = """
#include <condition_variable>
#include <iostream>
#include <mutex>
#include <thread>

#define N 5

enum {
  USING_RESOURCES     = 0, 
  IDLE                = 1, 
  READY_FOR_RESOURCES = 2,
};

enum {
  RESOURCE_FREE     = 0,
  RESOURCE_ACQUIRED = 1,
};

int user_state[N];
int resource_state[N];
std::pair<int, int> user_required_resources[N];

std::mutex mutex;
std::condition_variable resource_cvs[N];

void user_thread(int id) {
  for (int i = 0; i < 5; i++) {
    user_state[id] = IDLE;
    std::cout << id << " idle" << std::endl;

    // Simulate idle
    std::this_thread::sleep_for(std::chrono::milliseconds(1 + rand() % 1000));

    user_state[id] = READY_FOR_RESOURCES;
    std::cout << id << " ready for resources" << std::endl;
    
    std::unique_lock<std::mutex> lock(mutex);
    
    auto [x, y] = user_required_resources[id];

    std::cout << id << " waiting on " << x << std::endl;
    resource_cvs[x].wait(lock, [id, x] { return resource_state[x] != RESOURCE_ACQUIRED; });
    resource_state[x] = RESOURCE_ACQUIRED;
    std::cout << id << " acquired " << x << std::endl;

    std::cout << id << " waiting on " << y << std::endl;
    resource_cvs[y].wait(lock, [id, y] { return resource_state[y] != RESOURCE_ACQUIRED; });
    resource_state[y] = RESOURCE_ACQUIRED;
    std::cout << id << " acquired " << y << std::endl;

    user_state[id] = USING_RESOURCES;
    std::cout << id << " using resources" << std::endl;

    // Simulate using resources
    std::this_thread::sleep_for(std::chrono::milliseconds(1 + rand() % 1000));

    resource_state[x] = RESOURCE_FREE;
    std::cout << id << " freed " << x << std::endl;
    resource_cvs[x].notify_all();

    resource_state[y] = RESOURCE_FREE;
    std::cout << id << " freed " << y << std::endl;
    resource_cvs[y].notify_all();
  }
}

int main() {
  std::thread threads[N];

  for (int i = 0; i < N; i++) {
    user_required_resources[i] = {i, (i + 1) % N};
  }

  for (int i = 0; i < N; i++)
    threads[i] = std::thread(user_thread, i);

  for (int i = 0; i < N; i++)
    threads[i].join();

  return 0;
}
"""

In [3]:
prompt_no_hint = f"""
You are assisting with solving a violation present in the following code:

```
{before_code}
```

The violation is:

```yaml
description: |
  The following code uses multithreading. Inspect the code and verify its
  correctness.
```

Task:

Refactor the given code to solve the violation.

Ensure deprecated or removed APIs are replaced with valid alternatives.

Output:

Refactored Code

Step-by-Step Migration Process

New Hint Summary: One-paragraph explanation of what changed and why (to guide future similar migrations).

"""

result = await model_provider.ainvoke_llm(prompt_no_hint)

In [4]:
print("Prompt without hint:")
print(result.content)

Prompt without hint:
The code provided is a classic example of the "Dining Philosophers" problem, where multiple threads (users) are trying to acquire two resources (like forks) to perform a task. The main issue here is the potential for a deadlock, where each thread holds one resource and waits indefinitely for the other, causing the system to halt.

To solve this, we can implement a strategy to prevent deadlock by ensuring that all resources are acquired in a consistent order. This can be achieved by always acquiring the lower-numbered resource first. Additionally, we will replace deprecated or removed APIs if necessary, although in this case, the APIs used are current.

### Refactored Code

```cpp
#include <condition_variable>
#include <iostream>
#include <mutex>
#include <thread>

#define N 5

enum {
  USING_RESOURCES     = 0, 
  IDLE                = 1, 
  READY_FOR_RESOURCES = 2,
};

enum {
  RESOURCE_FREE     = 0,
  RESOURCE_ACQUIRED = 1,
};

int user_state[N];
int resource_stat

In [ ]:
prompt_with_hint = f"""
You are assisting with solving a violation present in the following code:

```
{before_code}
```

The violation is:

```yaml
description: |
  The following code uses multithreading. Inspect the code and verify its
  correctness.
hint: |
  This code uses multithreading. Think about the Chandy/Misra solution.
```

Task:

Refactor the given code to solve the violation.

Ensure deprecated or removed APIs are replaced with valid alternatives.

Output:

Refactored Code

Step-by-Step Migration Process

New Hint Summary: One-paragraph explanation of what changed and why (to guide future similar migrations).

"""

result = await model_provider.ainvoke_llm(prompt_with_hint)